In [9]:
from ctgan import CTGAN
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Carregar os dados originais
df = pd.read_csv('imputed_plataformas_otc.csv')

# Preencher valores ausentes (se houver)
df.fillna(df.mean(numeric_only=True), inplace=True)
df.fillna(0, inplace=True)  # Para colunas binárias

# Listas de colunas contínuas (floats) e colunas inteiras
colunas_float = ['weight (t)', 'height_of_jacket_or_sub-structure (m)', 'distance_to_coast (km)',
                 'energy_consumption-complete (GJ)', 'energy_consumption-partial (GJ)', 
                 'emissions-complete (t)', 'emissions-partial (t)']
colunas_inteiras = ['water_depth (m)', 'installation_date', 'type_of_production (1 oil and gas; 2 oil; 3 gas)', 'number_of_legs', 
                    'number_of_piles', 'recommended (1 partial; 2 complete)']

# Normalizar colunas contínuas (float)
scaler = MinMaxScaler()
df_normalizado = df.copy()
df_normalizado[colunas_float] = scaler.fit_transform(df[colunas_float])

# Instanciar o CTGAN
ctgan = CTGAN(epochs=2000, batch_size=3500)

# Treinar o CTGAN com os dados normalizados
ctgan.fit(df_normalizado)

# Gerar 1000 novos dados sintéticos
df_novos_dados_ctgan = ctgan.sample(1000)

# Reverter a normalização para os dados contínuos
df_novos_dados_ctgan[colunas_float] = scaler.inverse_transform(df_novos_dados_ctgan[colunas_float])

# Garantir que os dados sintéticos fiquem dentro do escopo dos dados originais
for coluna in colunas_float:
    min_val = df[coluna].min()
    max_val = df[coluna].max()
    df_novos_dados_ctgan[coluna] = df_novos_dados_ctgan[coluna].clip(lower=min_val, upper=max_val)

# Garantir que as colunas inteiras sejam arredondadas e respeitem os valores restritos
df_novos_dados_ctgan[colunas_inteiras] = df_novos_dados_ctgan[colunas_inteiras].round().astype(int)

# Garantir que as colunas restritas tenham valores corretos
df_novos_dados_ctgan['type_of_production (1 oil and gas; 2 oil; 3 gas)'] = df_novos_dados_ctgan['type_of_production (1 oil and gas; 2 oil; 3 gas)'].clip(1, 3)
df_novos_dados_ctgan['recommended (1 partial; 2 complete)'] = df_novos_dados_ctgan['recommended (1 partial; 2 complete)'].clip(1, 2)

# Salvar os novos dados sintéticos
df_novos_dados_ctgan.to_csv('novos_dados_sinteticos_ctgan.csv', index=False)

In [10]:
import pandas as pd
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

# Carregar os dados originais e sintéticos
df_original = pd.read_csv('imputed_plataformas_otc.csv')  # Seu dataset original
df_sintetico = pd.read_csv('novos_dados_sinteticos_ctgan.csv')  # Dados sintéticos gerados

metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "recommended (1 partial; 2 complete)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade entre os dados reais e sintéticos
report = QualityReport()
report.generate(real_data=df_original, synthetic_data=df_sintetico, metadata=metadata_dict)

# Exibir detalhes específicos do relatório
print(report.get_details(property_name='Column Shapes'))
print(report.get_details(property_name='Column Pair Trends'))


Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 13/13 [00:00<00:00, 1300.09it/s]|
Column Shapes Score: 82.85%

(2/2) Evaluating Column Pair Trends: |██████████| 78/78 [00:00<00:00, 207.74it/s]|
Column Pair Trends Score: 96.98%

Overall Score (Average): 89.92%

                                              Column        Metric   Score
0                                    water_depth (m)  KSComplement  0.7320
1                                         weight (t)  KSComplement  0.8435
2                                  installation_date  KSComplement  0.8085
3   type_of_production (1 oil and gas; 2 oil; 3 gas)  KSComplement  0.9185
4                                     number_of_legs  KSComplement  0.8810
5                                    number_of_piles  KSComplement  0.7770
6              height_of_jacket_or_sub-structure (m)  KSComplement  0.8540
7                             distance_to_coast (km)  KSComplement  0.8015
8                   energy_consumption-compl